#Model Architecture:
The implemented model leverages a Transformer architecture for sentiment analysis. The Transformer block consists of multiple self-attention mechanisms applied iteratively (num_heads). Each attention mechanism captures different aspects of the input sequence, allowing the model to efficiently learn complex patterns. A subsequent 1D convolutional layer is employed to capture local patterns, followed by global average pooling for dimensionality reduction. Dropout is used for regularization, and the final layers include a dense layer with ReLU activation and a sigmoid activation for binary sentiment classification.

##Step 1: Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Attention, LayerNormalization, Dropout

## Step 2: Load the Dataset
Reads the IMDB dataset from a CSV file using Pandas and stores it in a DataFrame (df).




In [ ]:
df = pd.read_csv("/content/IMDB Dataset.csv")

##Step 3: Data Cleaning and Preprocessing

* Cleans and preprocesses the text data in the 'review' column.
* Removes non-alphabetic characters.
* Converts text to lowercase.
* Removes stopwords using NLTK.

In [ ]:
df['cleaned_review'] = df['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
df['cleaned_review'] = df['cleaned_review'].apply(lambda x: x.lower())
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['cleaned_review'] = df['cleaned_review'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

## Step 4: Data Splitting
Splits the dataset into training and testing sets using the train_test_split function from scikit-learn.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42)

##Step 5: Label Encoding for Sentiment Column
Encodes the categorical 'sentiment' labels into numerical format using LabelEncoder.



In [ ]:
# Label Encoding for Sentiment Column
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

## Step 6: Tokenization and Padding
Tokenizes and pads the text sequences for input into a neural network using Keras's Tokenizer and pad_sequences.


In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_sequences, maxlen=100, truncating='post', padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, truncating='post', padding='post')

##Step 7: Transformer Model Building
Defines a Transformer model architecture using Keras. It includes an embedding layer, a Transformer block with attention mechanisms, convolutional and pooling layers, and final dense layers.


In [ ]:
# Transformer Model Building
embedding_dim = 32  # We can Adjust the embedding dimension based on our preference
num_heads = 2
ff_dim = 32

inputs = Input(shape=(100,))
embedding_layer = Embedding(input_dim=10000, output_dim=embedding_dim)(inputs)
transformer_block = embedding_layer

for _ in range(num_heads):
    attn_output = Attention(use_scale=True)([transformer_block, transformer_block])
    transformer_block = LayerNormalization(epsilon=1e-6)(transformer_block + attn_output)
    transformer_block = Dropout(0.1)(transformer_block)

transformer_block = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(transformer_block)
transformer_block = tf.keras.layers.GlobalAveragePooling1D()(transformer_block)
transformer_block = Dropout(0.1)(transformer_block)
transformer_block = Dense(20, activation='relu')(transformer_block)
output_layer = Dense(1, activation='sigmoid')(transformer_block)

model = Model(inputs=inputs, outputs=output_layer)

## Step 8: Model Training
Compiles and trains the Transformer model on the training data for 5 epochs.


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train_encoded, epochs=5, validation_data=(X_test_padded, y_test_encoded))

##Step 9: Save the Model
Saves the trained model in the HDF5 format for future use (while developing an app and its deployment)



In [ ]:
model.save('/content/transformer_sentiment_model.h5')

## Step 10: Evaluate the Model
Evaluates the model on the test set, computes accuracy, and generates a classification report.


In [ ]:
y_pred_prob = model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype(int)

# Converting back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred.flatten())

print("Accuracy:", accuracy_score(y_test_encoded, y_pred))
print("Classification Report:\n", classification_report(y_test_encoded, y_pred))

## Step 11: User Input Prediction
Defines a function for predicting sentiment based on user input and prompts the user to input a review for sentiment prediction.


In [ ]:
# User Input Prediction
def predict_sentiment_transformer(review):
    cleaned_review = re.sub(r'[^a-zA-Z\s]', '', review.lower())
    cleaned_review = ' '.join(word for word in cleaned_review.split() if word not in stop_words)
    sequence = tokenizer.texts_to_sequences([cleaned_review])
    padded_sequence = pad_sequences(sequence, maxlen=100, truncating='post', padding='post')
    prediction = model.predict(padded_sequence)
    if 0.4 <= prediction <= 0.55:
        return "Neutral"
    elif prediction > 0.55:
        return "Positive"
    else:
        return "Negative"

# Enter your review to get the prediction of the sentiment
user_input = input("Enter your Review here: ")
prediction = predict_sentiment_transformer(user_input)
print(f"Predicted Sentiment: {prediction}")